In [ ]:
import numpy as np
import openai
import pandas as pd
import os
import sys
import time
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

# Add the path to the constants file to the system path
sys.path.append('../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *
from CF_utils import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

In [ ]:
# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, 'data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'amazon-beauty/large_merged_data.csv')
print(f'Data path: {data_path}')

# zero shot save path
# CF_ZERO_SHOT_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/CF_large_predictions_zero_shot.csv')
# print(f'Zero shot save path: {CF_ZERO_SHOT_SAVE_PATH}')

# few shot save path
CF_FEW_SHOT_1_OBS_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/CF_large_1_test_predictions_few_shot.csv')
print(f'Few shot save path: {CF_FEW_SHOT_1_OBS_SAVE_PATH}')

In [ ]:
# Read the data
data = pd.read_csv(data_path)

# get statistic and first few data of NUM_SAMPLES rows
data.info()
data.head(NUM_EXAMPLES)


In [ ]:
%%time

# Example DataFrame columns
user_col = 'reviewerID'  # Replace with your user column name
item_col = 'asin'    # Replace with your item column name
rating_col = 'rating' # Replace with your rating column name

# Step 1: Create User-Item Interaction Matrix
interaction_matrix, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper = create_interaction_matrix(data, user_col, item_col, rating_col)

# Step 2: Fit the kNN Model
model_knn = fit_knn_model(interaction_matrix)

# Step 3: Make Recommendations for a specific user
user_id = 'ANV9L0JU6BNL'  # Replace with a user ID from your dataset
recommendations = recommend_items(user_id, interaction_matrix, user_mapper, item_inv_mapper, model_knn, n_recommendations=4)

print("Recommended Items:", recommendations)


# Collaborative filtering model to predict ratings - User based CF

In [ ]:
%%time

interaction_matrix, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper = create_interaction_matrix(data, user_col, item_col, rating_col)
model_knn = fit_knn_model(interaction_matrix)

predict_ratings_with_collaborative_filtering_and_save(data, interaction_matrix, user_mapper, item_mapper, user_inv_mapper, model_knn,
                                                      ITEM_SIDE + USER_SIDE + INTERACTION_SIDE, ITEM_SIDE,
                                                      user_column_name='reviewerID', title_column_name='title', asin_column_name='asin',
                                                      obs_per_user=1, save_path=CF_FEW_SHOT_1_OBS_SAVE_PATH)


In [ ]:
# Evaluate Few-shot Model
evaluate_model_predictions_rmse_mae(
    data_path=CF_FEW_SHOT_1_OBS_SAVE_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)